In [29]:
import numpy as np
from scipy.stats import skew, kurtosis

In [35]:
######################## Input parameters #########################
files = 200
case = 'case_a'
nclass = 20

In [36]:
points_per_file = 100000
data = np.zeros((points_per_file*files, 29))

######################## Read files and join them in only one array #########################
for j in range(files):
    data_file = np.load('../Results/model_eval/'+case+'/wu_eval_'+str(j)+".npy")
    data[j*points_per_file:(j+1)*points_per_file,:] = data_file

Sampling_Points = data[:,0:15]
output = 'k'
Results = data[:,18]*data[:,27]/(0.1*data[:,28])
# output = 'J'
# Results = data[:,18]

In [37]:
print('Mean ', np.mean(Results))
print('Var ', np.var(Results))
print('Skew ', skew(Results))
print('Kurt ', 3 + kurtosis(Results))

cond_mean=np.zeros((15,nclass))
cond_var=np.zeros((15,nclass))
cond_skew=np.zeros((15,nclass))
cond_kurt=np.zeros((15,nclass))

AMAE=np.zeros(15)
AMAV=np.zeros(15)
Si=np.zeros(15)
AMAskew=np.zeros(15)
AMAkurt=np.zeros(15)

Output = np.zeros((points_per_file*files,2))

Mean  1.0311170782908638e-13
Var  1.3764694354068975e-26
Skew  2.390186623344885
Kurt  29.48170414691884


In [38]:
for par in range(15):
    Output[:,0] = Sampling_Points[:,par]
    Output[:,1] = Results
    par_min = Sampling_Points[:,par].min()
    par_max = Sampling_Points[:,par].max()
    dpar = (par_max-par_min)/nclass

    for iclass in range (0,nclass):
        output_class = (Output[(Output[:,0]<par_min+dpar*(iclass+1)) & (Output[:,0]>par_min+dpar*iclass)])[:,1]
        cond_mean[par,iclass]=np.mean(output_class)
        cond_var[par,iclass]=np.var(output_class)
        cond_skew[par,iclass]=skew(np.reshape(output_class,-1))
        cond_kurt[par,iclass]=3+kurtosis(np.reshape(output_class,-1))

    Si[par]=(np.var(cond_mean[par,:],ddof=1))/np.var(Output[:,1])
    AMAE[par]=np.mean(abs(cond_mean[par,:]-np.mean(Output[:,1])))/abs(np.mean(Output[:,1]))
    AMAV[par]=np.mean(abs(cond_var[par,:]-np.var(Output[:,1])))/abs(np.var(Output[:,1]))
    AMAskew[par]=np.mean(abs(cond_skew[par,:]-skew(np.reshape(Output[:,1],-1))))/abs(skew(np.reshape(Output[:,1],-1)))
    AMAkurt[par]=np.mean(abs(cond_kurt[par,:]-(3+kurtosis(np.reshape(Output[:,1],-1)))))/abs((3+kurtosis(np.reshape(Output[:,1],-1))))

np.savetxt('../Results/SA/'+output+'/'+case+'/AMA.txt',(AMAE,AMAV,AMAskew,AMAkurt,Si))
np.savetxt('../Results/SA/'+output+'/'+case+'/CM.txt',cond_mean)
np.savetxt('../Results/SA/'+output+'/'+case+'/CV.txt',cond_var)
np.savetxt('../Results/SA/'+output+'/'+case+'/CS.txt',cond_skew)
np.savetxt('../Results/SA/'+output+'/'+case+'/CK.txt',cond_kurt)
np.savetxt('../Results/SA/'+output+'/'+case+'/SM.txt',[np.mean(Results), np.var(Results), skew(Results), 3 + kurtosis(Results)])